## Screenering reports

Below is an example of how you can quickly filter a corpus of PFD reports using natural language.

While the judiciary.uk website does offer a search function, it relies solely on keyword matching. Because coroners often refer to the same core concepts in different ways, this approach can miss relevant reports. Additionally, the judiciary.uk search only works for newer reports with embedded HTML; older reports are excluded, making comprehensive searching difficult. 

PFD Toolkit's Screener module offers greater flexibility and ensures access to the full archive.

First, we are going to load a small sample of reports to work from. While we're here, we'll also import the required `LLM` and `Screener` modules.

In [ ]:
%%time

from pfd_toolkit import load_reports, LLM, Screener
from dotenv import load_dotenv 
import os

reports = load_reports(start_date="2025-04-01", end_date="2025-05-25")
len(reports)

Next, we need to load our OpenAI API key and initialise our LLM client engine.

In [ ]:
%%time

# Load OpenAI API key
load_dotenv("api.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialise LLM client
llm_client = LLM(api_key=openai_api_key, max_workers=30, model="gpt-4o-mini")

Now, all we need to do is run our `filter_reports()` method, and it will comb through the provided catelogue, pulling out reports which meet the criteria.

Below, we're going to use a simple query: "suicide".

In [3]:
%%time

filter_object = Screener(llm=llm_client,
                        reports = reports,
                        user_query="suicide")

filtered_reports = filter_object.screen_reports()
filtered_reports

Sending requests to the LLM (in parallel): 100%|██████████| 71/71 [00:04<00:00, 16.20it/s]

CPU times: user 961 ms, sys: 33.3 ms, total: 994 ms
Wall time: 4.55 s


,URL,ID,Date,CoronerName,Area,Receiver,InvestigationAndInquest,CircumstancesOfDeath,MattersOfConcern
3,https://www.judiciary.uk/prevention-of-future-...,2025-0235,2025-05-20,Louise Hunt,Birmingham and Solihull,West Midlands Fire Service 1,On 5 February 2024 I commenced an investigatio...,Mr Brown was found hanging [REDACTED] at his h...,During the course of the inquest the evidence ...
6,https://www.judiciary.uk/prevention-of-future-...,2025-0234,2025-05-17,Sarah Murphy,Cheshire,The Royal College of General Practitioners (RC...,On 09 September 2024 I commenced an investigat...,The deceased was 28 years of age with a medica...,During the course of the investigation my inqu...
10,https://www.judiciary.uk/prevention-of-future-...,2025-0227,2025-05-13,Joanne Andrews,"West Sussex, Brighton and Hove",1 Sussex Partnership NHS Foundation Trust 2 NH...,On 08 March 2023 I commenced an investigation ...,Margaret Kagure Pauline Reece attended her GP ...,During the course of the investigation my inqu...
19,https://www.judiciary.uk/prevention-of-future-...,2025-0229,2025-05-08,Roland Wooderson,Gloucestershire,Department of Health and Social Care The Chief...,On 30 June 2023 I commenced an investigation i...,The deceased had a history of mental health di...,During the course of the inquest the evidence ...
27,https://www.judiciary.uk/prevention-of-future-...,2025-0211,2025-05-02,Victoria Davies,Cheshire,1 Governor for HMP Styal 2 Minister of State f...,On 24 July 2024 I commenced an investigation i...,Sarah Boyle was detained at HMP Styal between ...,During the course of the investigation my inqu...
33,https://www.judiciary.uk/prevention-of-future-...,2025-0120,2025-04-24,Samantha Marsh,Somerset,Part 1 1. Somerset Foundation Trust of Trust M...,On 6 th December 2022 I commenced an investiga...,Anne first presented to her GP in 2008. During...,4 During the course of the inquest the evidenc...
39,https://www.judiciary.uk/prevention-of-future-...,2025-0120,2025-04-17,Sean Horstead,Essex,Essex County Council,On 24 th November 2023 I commenced an investig...,Linda's husband had been suffering from signif...,During the inquest the evidence revealed matte...
51,https://www.judiciary.uk/prevention-of-future-...,2025-0184,2025-04-10,Lydia Brown,West London,"1. Chief Executive, South West London and St G...",On 30 August I opened 2023 an investigation in...,Jonathan had been under the care of mental hea...,During the course of the inquest the evidence ...
59,https://www.judiciary.uk/prevention-of-future-...,2025-0172,2025-04-07,Sian Reeves,South London,1. South London and Maudsley NHS Foundation Tr...,"On 21 March 2023, an inquest was opened, and a...",Christopher McDonald was pronounced dead at 14...,During the course of the inquest the evidence ...
67,https://www.judiciary.uk/prevention-of-future-...,2025-0167,2025-04-03,Kate Bisset,Lancashire and Blackburn with Darwen,NHS Pathways,On the 3 rd April 2024 the Coroner's Office wa...,Mr James Masheter was a 42-year-old man with a...,During the course of the inquest the evidence ...


The result is a smaller dataframe, containing reports which meet the user query.

### Further options

In this example, we're filtering by a broader (and wordier!) query. We want to pull out reports which mention some form of medication.

We've also customised our output through a series of additional parameters...
* `include_*` - we can toggle any PFD reports section on or off. If our query relates to one specific section (Circumstances of Death), then we don't really need the model to see the Investigation or Concerns sections. By default, the shorter sections are also turned off (i.e. receiver, coroner name, area) but you can manually turn these on too.
* `match_leniency` - either *strict* or *liberal* (defaults to *strict*). If the AI is on the fence whether a report is relevant to your query, you can decide to exclude it (strict) or include it (liberal) in the final dataframe.
* `filter_df` - either *True* or *False* (defaults to *True*). Whether to filter the dataframe so that only relevant reports are returned (*True*), or whether to return a column which indicates whether the report fulfils the query or not.
* `result_col_name` - if above is set to False, you can optionally denote the column name you want added to your dataframe.

In [4]:
%%time

user_query = "Reports which explicitly mention death from a specific medication."

advanced_filter = Screener(reports=reports, 
                         llm=llm_client,
                         user_query=user_query, 
                        
                         include_circumstances=True,
                         include_investigation=False,
                         include_concerns=False,
                         
                         match_leniency='liberal', 
                         filter_df=False,
                         result_col_name='medication_error')

medication_reports = advanced_filter.screen_reports().reset_index(drop=True)
medication_reports


Sending requests to the LLM (in parallel): 100%|██████████| 71/71 [00:05<00:00, 13.91it/s]

CPU times: user 867 ms, sys: 68 ms, total: 935 ms
Wall time: 5.39 s


,URL,ID,Date,CoronerName,Area,Receiver,InvestigationAndInquest,CircumstancesOfDeath,MattersOfConcern,medication_error
0,https://www.judiciary.uk/prevention-of-future-...,2025-0237,2025-05-21,Guy Davies,Cornwall and the Isles of Scilly,Interim Chief Executive NHS University Hospita...,On 30 July 2024 I commenced an investigation i...,Dave opted for an elective surgical procedure ...,During the course of the inquest the evidence ...,False
1,https://www.judiciary.uk/prevention-of-future-...,2025-0236,2025-05-21,Kate Robertson,North West Wales,Betsi Cadwaladr University Health Board (BCUHB) 1,On 20 May 2024 I commenced an investigation in...,The circumstances of the death are as follows ...,"During the course of the inquest, the evidence...",False
2,https://www.judiciary.uk/prevention-of-future-...,2025-0238,2025-05-21,Caroline Saunders,Gwent,The Chief Executive of Aneurin Bevan Universit...,On 28/2/2024 an investigation was opened touch...,Marina Waldron was admitted to hospital on 17/...,The MATTERS OF CONCERN are as follows: - Durin...,False
3,https://www.judiciary.uk/prevention-of-future-...,2025-0235,2025-05-20,Louise Hunt,Birmingham and Solihull,West Midlands Fire Service 1,On 5 February 2024 I commenced an investigatio...,Mr Brown was found hanging [REDACTED] at his h...,During the course of the inquest the evidence ...,False
4,https://www.judiciary.uk/prevention-of-future-...,2025-0232,2025-05-19,Edward Steele,East Riding of Yorkshire and City of Kingston ...,"1. Holderness Health, Hedon Group Practice. 2....","On 18 June 2024, I commenced an investigation ...",Mr Spencer became unwell four days before his ...,During the course of the inquest the evidence ...,False
...,...,...,...,...,...,...,...,...,...,...
66,https://www.judiciary.uk/prevention-of-future-...,2025-0210,2025-04-03,Alison Hewitt,City of London,"1. , Company Secretary of Samba London Limited...",I have commenced an investigation into the dea...,"On the evidence currently available to me, I u...",The evidence I have gathered to date reveals m...,False
67,https://www.judiciary.uk/prevention-of-future-...,2025-0167,2025-04-03,Kate Bisset,Lancashire and Blackburn with Darwen,NHS Pathways,On the 3 rd April 2024 the Coroner's Office wa...,Mr James Masheter was a 42-year-old man with a...,During the course of the inquest the evidence ...,False
68,https://www.judiciary.uk/prevention-of-future-...,2025-0174,2025-04-03,Guy Davies,Cornwall and the Isles of Scilly,", Secretary of State for Health and Social Care 1",On 30 May 2024 I commenced an investigation in...,1. The findings of fact on how Andrew died are...,During the course of the inquest the evidence ...,False
69,https://www.judiciary.uk/prevention-of-future-...,2025-0178,2025-04-03,Crispin Oliver,County Durham and Darlington,REDACTED,On 26/05/2023 14:31an investigation was commen...,"Died at [REDACTED], Darlington, on 13 May 2023...",During the course of the investigation my inqu...,False
